In [2]:
from turtle import width
import cv2
import mediapipe as mp

# 얼굴을 찾고, 찾은 얼굴에 표시를 해주기 위한 변수 정의
mp_face_detection = mp.solutions.face_detection         # 얼굴 검출을 위한 face_detection 모듈을 사용
mp_drawing = mp.solutions.drawing_utils                 # 얼굴의 특징을 그리기 위한 drawing_utils 모듈을 사용


cap = cv2.VideoCapture(0)



# model_selection : face-dection 객체를 만들 때 상황(0 : 1 = 근거리영상 : 원거리영상 )에 맞는 영상에 따라 선택
# min_detection_confidence : 얼굴로 판단할 어느정도의 신뢰도를 정의 (treshold와 비슷한 개념)
with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.7) as face_detection:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_detection.process(image)                 # process : 이미지로부터 얼굴을 검출해 결과를 반환하는 함수 

        # Draw the face detection annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        image = cv2.resize(image, (1920,1080))
        
        if results.detections:
            # 6개의 특징을 추출 : 우좌/눈, 코 끝, 입중심, 우좌/귀( 귀구슬점, 이주 )
            for detection in results.detections:
                mp_drawing.draw_detection(image, detection)     # 얼굴이라고 인식하는 6부분에 점을 찍기
                # print(detection)
                
                h, w, _ = image.shape
                
                keypoints = detection.location_data.relative_bounding_box
                x = int(keypoints.xmin * w)
                y = int(keypoints.ymin * h)
                w = int(keypoints.width * w)
                h = int(keypoints.height * h)
                
                crop = image[y:y+h, x :x+w]
                
                
                image[y:y+h, x :x+w] = cv2.GaussianBlur(crop, (0,0), 33)
                      
                
        # Flip the image horizontally for a selfie-view display.
        cv2.imshow('MediaPipe Face Detection', cv2.resize(image, None, fx=0.5, fy =0.5))
        
        if cv2.waitKey(5) == ord('q'):
            print(image.shape)
            break
        
cap.release()
cv2.destroyAllWindows()

(1080, 1920, 3)
